In [1]:
import datetime
import time
import math
import pandas as pd
import numpy as np
import scipy.stats as stat
from numpy import std, mean
#import statsmodels.api as sm
import statsmodels.stats as sm_stats
from statsmodels.stats.proportion import proportions_ztest
import ast
import re, string, unicodedata
import copy
import pyodbc
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date, datetime
from dateutil.relativedelta import relativedelta

from sklearn.model_selection import train_test_split
from sklearn.base import RegressorMixin, BaseEstimator, ClassifierMixin
from sklearn.metrics import average_precision_score
from catboost import CatBoostClassifier, CatBoostRegressor, to_classifier, EFstrType
import shap
shap.initjs() 

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame
from pyspark.sql import Window
from pyspark.sql import functions as f 
from pyspark.sql.functions import row_number
from pyspark.sql import types
from pyspark.sql.types import *
from pyspark import SparkConf
from pyspark.sql.functions import udf
from pyspark.sql.functions import col, countDistinct,concat, isnan, when, count
import os
import subprocess

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

In [3]:
#Create the connection to hadoop cluster
appname='abc'
spark = (SparkSession.builder.appName(appname)
.enableHiveSupport().getOrCreate())

### Assign Variables

In [4]:
#Cohort and related dates
MMYYYY = '072022'
cohort = '2022-07-31'

### Load data and convert Spark DF to Pandas DF

In [ ]:
#Pull data into spark DF and convert to pandas DF
data = spark.sql("""
select * from 
""".format(MMYYYY=MMYYYY))

decimal_cols = [ f[0] for f in data.dtypes if f[1].startswith('decimal') ]
data = data.select(
    [col(name).cast('double') if name in decimal_cols else col(name) for name, colType in data.dtypes]
)

data.createOrReplaceTempView("data")

df = data.toPandas()


In [ ]:
#list of useful features, with some removed (i.e. constant features), and a list of features with collinear features removed
features_list = [
    ''
    ,''
]

collinear_features_to_remove = [
    ''
    ,''
]

features_list_decollinear = [feature for feature in features_list if feature not in collinear_features_to_remove]

## Propensity Modeling

In [ ]:
#define cat features
cat_features = [
    j for j, name in 
    enumerate(df[features_list_decollinear]) if not pd.api.types.is_numeric_dtype(df[name])
]

In [10]:
#Using cross validation, extract models for averaged prediction
from catboost import Pool, cv

# x, t, idx = df[features_list], df['treatment_status'], df.index
x, t, idx = df[features_list_decollinear], df['treatment_status'], df.index

cv_pool = Pool(data=x,
                  label=t,
                  cat_features=cat_features)

#Use PRAUC as metric due to high imbalance of the data (~2.4% positive class)
params = {"iterations": 1000,
          "depth": 3,
          "loss_function": "Logloss",
          "verbose": True,
          "early_stopping_rounds": 25,
          "random_state": 2023,
          "eval_metric": "PRAUC"}

scores = cv(cv_pool,
            params,
            fold_count=3, 
            plot="True",
            as_pandas=True,
            stratified=True,
            return_models=True)

#From CV results we see that models converge with a PRAUC average of ~0.45, which is significantly better than the positive class
#prevalence of ~0.02


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Training on fold [0/3]
0:	learn: 0.4094519	test: 0.4064341	best: 0.4064341 (0)	total: 290ms	remaining: 4m 50s
1:	learn: 0.4103128	test: 0.4054458	best: 0.4064341 (0)	total: 528ms	remaining: 4m 23s
2:	learn: 0.3886289	test: 0.3835227	best: 0.4064341 (0)	total: 766ms	remaining: 4m 14s
3:	learn: 0.4052043	test: 0.3995903	best: 0.4064341 (0)	total: 1.03s	remaining: 4m 16s
4:	learn: 0.4170340	test: 0.4114611	best: 0.4114611 (4)	total: 1.28s	remaining: 4m 15s
5:	learn: 0.4198367	test: 0.4148545	best: 0.4148545 (5)	total: 1.53s	remaining: 4m 13s
6:	learn: 0.4175512	test: 0.4124096	best: 0.4148545 (5)	total: 1.78s	remaining: 4m 11s
7:	learn: 0.4194267	test: 0.4146723	best: 0.4148545 (5)	total: 2.03s	remaining: 4m 11s
8:	learn: 0.4201941	test: 0.4154000	best: 0.4154000 (8)	total: 2.29s	remaining: 4m 12s
9:	learn: 0.4209475	test: 0.4164327	best: 0.4164327 (9)	total: 2.53s	remaining: 4m 10s
10:	learn: 0.4240908	test: 0.4187532	best: 0.4187532 (10)	total: 2.77s	remaining: 4m 8s
11:	learn: 0.424640

93:	learn: 0.4478562	test: 0.4435618	best: 0.4435618 (93)	total: 26.3s	remaining: 4m 13s
94:	learn: 0.4479320	test: 0.4436182	best: 0.4436182 (94)	total: 26.6s	remaining: 4m 13s
95:	learn: 0.4479869	test: 0.4436472	best: 0.4436472 (95)	total: 26.9s	remaining: 4m 13s
96:	learn: 0.4480931	test: 0.4437417	best: 0.4437417 (96)	total: 27.2s	remaining: 4m 12s
97:	learn: 0.4481843	test: 0.4438209	best: 0.4438209 (97)	total: 27.5s	remaining: 4m 12s
98:	learn: 0.4482656	test: 0.4439047	best: 0.4439047 (98)	total: 27.8s	remaining: 4m 12s
99:	learn: 0.4483831	test: 0.4440167	best: 0.4440167 (99)	total: 28.1s	remaining: 4m 13s
100:	learn: 0.4484764	test: 0.4441444	best: 0.4441444 (100)	total: 28.4s	remaining: 4m 13s
101:	learn: 0.4485533	test: 0.4442113	best: 0.4442113 (101)	total: 28.8s	remaining: 4m 13s
102:	learn: 0.4486751	test: 0.4443517	best: 0.4443517 (102)	total: 29.1s	remaining: 4m 13s
103:	learn: 0.4487080	test: 0.4443939	best: 0.4443939 (103)	total: 29.4s	remaining: 4m 13s
104:	learn: 0

185:	learn: 0.4536250	test: 0.4485203	best: 0.4485203 (185)	total: 54.6s	remaining: 3m 59s
186:	learn: 0.4536578	test: 0.4485355	best: 0.4485355 (186)	total: 54.9s	remaining: 3m 58s
187:	learn: 0.4537381	test: 0.4486182	best: 0.4486182 (187)	total: 55.2s	remaining: 3m 58s
188:	learn: 0.4537938	test: 0.4486686	best: 0.4486686 (188)	total: 55.5s	remaining: 3m 58s
189:	learn: 0.4538277	test: 0.4486897	best: 0.4486897 (189)	total: 55.8s	remaining: 3m 57s
190:	learn: 0.4538477	test: 0.4486908	best: 0.4486908 (190)	total: 56.1s	remaining: 3m 57s
191:	learn: 0.4539121	test: 0.4487524	best: 0.4487524 (191)	total: 56.4s	remaining: 3m 57s
192:	learn: 0.4539429	test: 0.4487809	best: 0.4487809 (192)	total: 56.7s	remaining: 3m 57s
193:	learn: 0.4539643	test: 0.4487928	best: 0.4487928 (193)	total: 57s	remaining: 3m 56s
194:	learn: 0.4540953	test: 0.4489009	best: 0.4489009 (194)	total: 57.3s	remaining: 3m 56s
195:	learn: 0.4541528	test: 0.4489495	best: 0.4489495 (195)	total: 57.6s	remaining: 3m 56s
1

275:	learn: 0.4576026	test: 0.4516375	best: 0.4516375 (275)	total: 1m 22s	remaining: 3m 37s
276:	learn: 0.4576322	test: 0.4516388	best: 0.4516388 (276)	total: 1m 23s	remaining: 3m 37s
277:	learn: 0.4577120	test: 0.4516866	best: 0.4516866 (277)	total: 1m 23s	remaining: 3m 37s
278:	learn: 0.4577252	test: 0.4516958	best: 0.4516958 (278)	total: 1m 23s	remaining: 3m 36s
279:	learn: 0.4577566	test: 0.4517249	best: 0.4517249 (279)	total: 1m 24s	remaining: 3m 36s
280:	learn: 0.4578011	test: 0.4517844	best: 0.4517844 (280)	total: 1m 24s	remaining: 3m 36s
281:	learn: 0.4578210	test: 0.4518038	best: 0.4518038 (281)	total: 1m 24s	remaining: 3m 35s
282:	learn: 0.4578651	test: 0.4518383	best: 0.4518383 (282)	total: 1m 25s	remaining: 3m 35s
283:	learn: 0.4578846	test: 0.4518379	best: 0.4518383 (282)	total: 1m 25s	remaining: 3m 35s
284:	learn: 0.4579192	test: 0.4518683	best: 0.4518683 (284)	total: 1m 25s	remaining: 3m 34s
285:	learn: 0.4579474	test: 0.4518913	best: 0.4518913 (285)	total: 1m 25s	remain

...

539:	learn: 0.4613961	test: 0.4569989	best: 0.4570322 (536)	total: 2m 38s	remaining: 2m 14s
540:	learn: 0.4614114	test: 0.4570108	best: 0.4570322 (536)	total: 2m 38s	remaining: 2m 14s
541:	learn: 0.4614372	test: 0.4570325	best: 0.4570325 (541)	total: 2m 38s	remaining: 2m 14s
542:	learn: 0.4614502	test: 0.4570307	best: 0.4570325 (541)	total: 2m 39s	remaining: 2m 14s
543:	learn: 0.4614737	test: 0.4570396	best: 0.4570396 (543)	total: 2m 39s	remaining: 2m 13s
544:	learn: 0.4614903	test: 0.4570339	best: 0.4570396 (543)	total: 2m 39s	remaining: 2m 13s
545:	learn: 0.4615081	test: 0.4570498	best: 0.4570498 (545)	total: 2m 40s	remaining: 2m 13s
546:	learn: 0.4615432	test: 0.4570700	best: 0.4570700 (546)	total: 2m 40s	remaining: 2m 12s
547:	learn: 0.4615581	test: 0.4570599	best: 0.4570700 (546)	total: 2m 40s	remaining: 2m 12s
548:	learn: 0.4615702	test: 0.4570556	best: 0.4570700 (546)	total: 2m 41s	remaining: 2m 12s
549:	learn: 0.4615815	test: 0.4570561	best: 0.4570700 (546)	total: 2m 41s	remain

629:	learn: 0.4630304	test: 0.4576680	best: 0.4576736 (626)	total: 3m 6s	remaining: 1m 49s
630:	learn: 0.4630449	test: 0.4576673	best: 0.4576736 (626)	total: 3m 6s	remaining: 1m 49s
631:	learn: 0.4630748	test: 0.4577040	best: 0.4577040 (631)	total: 3m 6s	remaining: 1m 48s
632:	learn: 0.4630886	test: 0.4577070	best: 0.4577070 (632)	total: 3m 7s	remaining: 1m 48s
633:	learn: 0.4631252	test: 0.4577260	best: 0.4577260 (633)	total: 3m 7s	remaining: 1m 48s
634:	learn: 0.4631411	test: 0.4577242	best: 0.4577260 (633)	total: 3m 7s	remaining: 1m 47s
635:	learn: 0.4631480	test: 0.4577243	best: 0.4577260 (633)	total: 3m 8s	remaining: 1m 47s
636:	learn: 0.4631571	test: 0.4577294	best: 0.4577294 (636)	total: 3m 8s	remaining: 1m 47s
637:	learn: 0.4631833	test: 0.4577529	best: 0.4577529 (637)	total: 3m 8s	remaining: 1m 47s
638:	learn: 0.4631999	test: 0.4577537	best: 0.4577537 (638)	total: 3m 9s	remaining: 1m 46s
639:	learn: 0.4632213	test: 0.4577781	best: 0.4577781 (639)	total: 3m 9s	remaining: 1m 46s

719:	learn: 0.4645359	test: 0.4581590	best: 0.4581590 (719)	total: 3m 33s	remaining: 1m 23s
720:	learn: 0.4645481	test: 0.4581536	best: 0.4581590 (719)	total: 3m 34s	remaining: 1m 22s
721:	learn: 0.4645571	test: 0.4581571	best: 0.4581590 (719)	total: 3m 34s	remaining: 1m 22s
722:	learn: 0.4645761	test: 0.4581632	best: 0.4581632 (722)	total: 3m 34s	remaining: 1m 22s
723:	learn: 0.4645806	test: 0.4581633	best: 0.4581633 (723)	total: 3m 35s	remaining: 1m 21s
724:	learn: 0.4645971	test: 0.4581780	best: 0.4581780 (724)	total: 3m 35s	remaining: 1m 21s
725:	learn: 0.4646132	test: 0.4581829	best: 0.4581829 (725)	total: 3m 35s	remaining: 1m 21s
726:	learn: 0.4646301	test: 0.4581844	best: 0.4581844 (726)	total: 3m 35s	remaining: 1m 21s
727:	learn: 0.4646567	test: 0.4581894	best: 0.4581894 (727)	total: 3m 36s	remaining: 1m 20s
728:	learn: 0.4646707	test: 0.4581895	best: 0.4581895 (728)	total: 3m 36s	remaining: 1m 20s
729:	learn: 0.4646860	test: 0.4581998	best: 0.4581998 (729)	total: 3m 36s	remain

809:	learn: 0.4658345	test: 0.4584100	best: 0.4584155 (807)	total: 4m 2s	remaining: 56.8s
810:	learn: 0.4658429	test: 0.4584115	best: 0.4584155 (807)	total: 4m 2s	remaining: 56.5s
811:	learn: 0.4658586	test: 0.4584315	best: 0.4584315 (811)	total: 4m 2s	remaining: 56.2s
812:	learn: 0.4658688	test: 0.4584235	best: 0.4584315 (811)	total: 4m 3s	remaining: 55.9s
813:	learn: 0.4658814	test: 0.4584243	best: 0.4584315 (811)	total: 4m 3s	remaining: 55.6s
814:	learn: 0.4658966	test: 0.4584338	best: 0.4584338 (814)	total: 4m 3s	remaining: 55.3s
815:	learn: 0.4659167	test: 0.4584087	best: 0.4584338 (814)	total: 4m 4s	remaining: 55.1s
816:	learn: 0.4659277	test: 0.4584057	best: 0.4584338 (814)	total: 4m 4s	remaining: 54.8s
817:	learn: 0.4659379	test: 0.4584085	best: 0.4584338 (814)	total: 4m 5s	remaining: 54.5s
818:	learn: 0.4659434	test: 0.4584013	best: 0.4584338 (814)	total: 4m 5s	remaining: 54.3s
819:	learn: 0.4659493	test: 0.4583902	best: 0.4584338 (814)	total: 4m 5s	remaining: 54s
820:	learn: 

900:	learn: 0.4671092	test: 0.4586610	best: 0.4586800 (896)	total: 5m 43s	remaining: 37.7s
901:	learn: 0.4671119	test: 0.4586628	best: 0.4586800 (896)	total: 5m 45s	remaining: 37.5s
902:	learn: 0.4671260	test: 0.4586619	best: 0.4586800 (896)	total: 5m 46s	remaining: 37.3s
903:	learn: 0.4671446	test: 0.4586696	best: 0.4586800 (896)	total: 5m 48s	remaining: 37s
904:	learn: 0.4671515	test: 0.4586699	best: 0.4586800 (896)	total: 5m 49s	remaining: 36.7s
905:	learn: 0.4671690	test: 0.4586692	best: 0.4586800 (896)	total: 5m 50s	remaining: 36.4s
906:	learn: 0.4671719	test: 0.4586697	best: 0.4586800 (896)	total: 5m 51s	remaining: 36.1s
907:	learn: 0.4671820	test: 0.4586751	best: 0.4586800 (896)	total: 5m 53s	remaining: 35.8s
908:	learn: 0.4671990	test: 0.4586868	best: 0.4586868 (908)	total: 5m 54s	remaining: 35.5s
909:	learn: 0.4672049	test: 0.4586958	best: 0.4586958 (909)	total: 5m 55s	remaining: 35.2s
910:	learn: 0.4672240	test: 0.4587189	best: 0.4587189 (910)	total: 5m 57s	remaining: 34.9s
9

In [11]:
#get SHAP values for each model

model_1_shap = scores[1][0].get_feature_importance(data=cv_pool, type=EFstrType.ShapValues)
model_2_shap = scores[1][1].get_feature_importance(data=cv_pool, type=EFstrType.ShapValues)
model_3_shap = scores[1][2].get_feature_importance(data=cv_pool, type=EFstrType.ShapValues)

In [5]:
#shap_values = model.get_feature_importance(Pool(X, y), type='ShapValues')

# summarize the effects of all the features for CV model 2

expected_value = model_2_shap[0,-1]
shap_values = model_2_shap[:,:-1]
shap.summary_plot(shap_values, x)

In [6]:
# summarize the effects of all the features for CV model 1

expected_value = model_1_shap[0,-1]
shap_values = model_1_shap[:,:-1]
shap.summary_plot(shap_values, x)

In [7]:
# summarize the effects of all the features for CV model 3

expected_value = model_3_shap[0,-1]
shap_values = model_3_shap[:,:-1]
shap.summary_plot(shap_values, x)

### Obtaining Propensity via averaging CV model scores

In [15]:
#Average out each CV model prediction as the final predict_proba
cv_models = scores[1]
cv_models_predict_proba = []

for model in list(range(0,3)):
    cv_models[model] = to_classifier(cv_models[model])
    pred = cv_models[model].predict_proba(x)
    cv_models_predict_proba.append(pred)
    

In [16]:
#assign average propensity (g_hat) and add to dataframe
g_hat = (cv_models_predict_proba[0][:,1] + cv_models_predict_proba[1][:,1] + cv_models_predict_proba[2][:,1])/3

df['g_hat'] = g_hat

## Propensity Methods for Causal Inference

### Matching

In [14]:
import propensity_functions

In [15]:
#set seed here for reproducibility of results
np.random.seed(2023)

#Assign groups and propensities
treatment = df.treatment_status
pscore = df.g_hat
g1_1_wor = Match(treatment,pscore)

In [20]:
#create the match object. 

"""
Note here the "caliper" hyperparameter is the percentage of the propensity score's (or logit of propensity score's) standard deviation to use.
caliper = 0.2, caliper_scale = 'logit' means the 0.2*(std.dev of the propensity score logit) will be used as the caliper size.
This size is recommended following (Austin, 2014)'s meta analysis (https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4285163/)

"""


"""
The problem with the above approach is that the derived caliper value is around .28, or 28% propensity error. Given the imbalance in our data and the size of the non-treatment group,
we get a better match result with a much smaller caliper (i.e 1% of actual propensity)

"""

#Smaller Caliper (i.e 1% score difference)
g1_1_wor.create(method='one-to-one', caliper_scale='propensity', caliper=0.01, replace=False)

In [24]:
##Created matched dataframe, report feature mean and count

data_matched = whichMatched(g1_1_wor, df, show_duplicates = False)

# check how many instances were successfully matched
# data_matched.pivot_table(values=features_list,
#                          index='treatment_status',
#                          aggfunc=[np.mean, np.size],
#                          margins=True
# )

# Assessing Match Quality

### Visual Inspection

In [15]:
#some features need to be log-scaled for better visibility 
log_kde_features = [
    ''
    ]

kde_features = [feature for feature in features_list if feature not in log_kde_features]


In [4]:
#Numeric features requiring a log scale -replace 0 with 0.1 for plotting purposes as log_scale cannot divide by 0
df_test = df
df_test[log_kde_features] = df_test[log_kde_features].replace(0,0.1)

data_matched_test = data_matched
data_matched_test[log_kde_features] = data_matched_test[log_kde_features].replace(0,0.1)

for feature in log_kde_features:
    comparison_plot_hist(df_test,data_matched_test,feature, log_scale=True)

In [2]:
#Numeric features not requiring a log scale
for feature in kde_features:
    comparison_plot_hist(df,data_matched,feature)

In [3]:
comparison_plot_cat(df,data_matched,'')

### Statistical Tests to Assess Matching Quality

#### Note: to assess informational differences of features, KL divergence or Population Stability Index may be more suitable

In [8]:
#Stat Test Pvalues, before and after matching

#Before Matching

for col in features_list:
    x=np.array(df[df['treatment_status']==1][col])
    y=np.array(df[df['treatment_status']==0][col])
    if np.array_equal(x, x.astype(bool)) & np.array_equal(y, y.astype(bool)):
        count = np.array([np.sum(x), np.sum(y)])
        nobs = np.array([x.size, y.size])
        a, pval2 = proportions_ztest(count, nobs)
        print('feature: ', col)
        print('proportions_ztest pval: ', pval2)
        print('*************************************************************************************************')
    else:
        pval=ttest_ind(x, y,nan_policy='omit').pvalue
        wilcoxon_ranksum_pval=ranksums(x, y).pvalue
        ks_2samp_pval=stat.ks_2samp(x, y, alternative='two-sided').pvalue
    #print(col,'- T-Test/proportion ztest: ',pval, '','*** Wilcoxon Rank Sum:',wilcoxon_ranksum_pval, '*** KS GoF Test',ks_2samp_pval)
    #for excel copying
    print('feature: ', col)
    print('ttest_ind pval: ', pval, ',','wilcoxon_ranksum_pval: ', wilcoxon_ranksum_pval,',','ks_2samp_pval: ', ks_2samp_pval)
    print('*************************************************************************************************')

In [9]:
#After Matching
for col in features_list:
    x=np.array(data_matched[data_matched['treatment_status']==1][col])
    y=np.array(data_matched[data_matched['treatment_status']==0][col])
    if np.array_equal(x, x.astype(bool)) & np.array_equal(y, y.astype(bool)):
        count = np.array([np.sum(x), np.sum(y)])
        nobs = np.array([x.size, y.size])
        a, pval2 = proportions_ztest(count, nobs)
        print('feature: ', col)
        print('proportions_ztest pval: ', pval2)
        print('*************************************************************************************************')
    else:
        pval=ttest_ind(x, y,nan_policy='omit').pvalue
        wilcoxon_ranksum_pval=ranksums(x, y).pvalue
        ks_2samp_pval=stat.ks_2samp(x, y, alternative='two-sided').pvalue
    #print(col,'- T-Test/proportion ztest: ',pval, '','*** Wilcoxon Rank Sum:',wilcoxon_ranksum_pval, '*** KS GoF Test',ks_2samp_pval)
    #for excel copying
    print('feature: ', col)
    print('ttest_ind pval: ', pval, ',','wilcoxon_ranksum_pval: ', wilcoxon_ranksum_pval,',','ks_2samp_pval: ', ks_2samp_pval)
    print('*************************************************************************************************')

### Significance Testing using Propensity Matched Data

In [85]:
x=np.array(data_matched[data_matched['treatment_status']==1]['label'])
y=np.array(data_matched[data_matched['treatment_status']==0]['label'])
count = np.array([np.sum(x), np.sum(y)])
nobs = np.array([x.size, y.size])
a, pval2 = proportions_ztest(count, nobs)

print(pval2)

0.3579014656696514
